<a href="https://colab.research.google.com/github/diogenesjusto/FIAP/blob/master/Gradua%C3%A7%C3%A3o/2022/S1/20220913_Bagging_Boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [58]:
df = pd.read_csv("https://raw.githubusercontent.com/diogenesjusto/FIAP/master/dados/train_titanic.csv")

# Transformações de dados: 1) criação de dummies 2) preenchimento de nulos idade com média de idade por classe
du_Sex = pd.get_dummies(df['Sex'])
du_Pclass = pd.get_dummies(df['Pclass'], prefix='class')
dtAge = df['Age'].fillna(df.groupby('Pclass')['Age'].transform('mean'))

# União das variáves no dataset de trabalho
df = df.join(du_Sex)
df = df.join(du_Pclass)
df['AgeFilledNa'] = dtAge


In [32]:
for i in range(10):
  x_treino, x_teste, y_treino, y_teste = train_test_split(
      df[['female','class_1','class_2','SibSp','Parch', 'AgeFilledNa']],
      df['Survived'], test_size=0.326, random_state=i)

  mod = RandomForestClassifier(n_estimators=500, bootstrap=True, random_state=1)
  mod = mod.fit(x_treino, y_treino)
  y_prev = mod.predict(x_teste)
  acuracia_bootstrap_true = accuracy_score(y_teste, y_prev)

  mod = RandomForestClassifier(n_estimators=1000, bootstrap=False, random_state=1)
  mod = mod.fit(x_treino, y_treino)
  y_prev = mod.predict(x_teste)
  acuracia_bootstrap_false = accuracy_score(y_teste, y_prev)

  print(acuracia_bootstrap_true, acuracia_bootstrap_false)

0.7903780068728522 0.7697594501718213
0.7491408934707904 0.738831615120275
0.7938144329896907 0.7663230240549829
0.7972508591065293 0.7938144329896907
0.8075601374570447 0.7972508591065293
0.8041237113402062 0.7903780068728522
0.8178694158075601 0.8178694158075601
0.7766323024054983 0.7766323024054983
0.8144329896907216 0.8075601374570447
0.8075601374570447 0.8041237113402062


In [33]:
# Instalação CatBoost
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 61.8 MB/s 


In [45]:
from xgboost import XGBClassifier
import lightgbm as lgb
import catboost as ctb
from catboost.core import CatBoostClassifier
import time

In [59]:
classificadores = [
    XGBClassifier(),
    lgb.LGBMClassifier(),
    ctb.CatBoostClassifier()]

nomes_classificadores = ['XGBoost','LightGBM', 'CatBoost']

# Aumento 'artificial' do tamanho da base
for i in range(5):
  df = pd.concat([df,df])

x_treino, x_teste, y_treino, y_teste = train_test_split(
      df[['female','class_1','class_2','SibSp','Parch', 'AgeFilledNa']],
      df['Survived'], test_size=0.326, random_state=2)
 
for idx, clf in enumerate(classificadores):
  # Marcação de tempo de início
  inicio = time.time()
  if nomes_classificadores[idx]=='CatBoost': 
    clf = CatBoostClassifier(iterations=20, learning_rate=0.15, depth=10)
    mod = clf.fit(x_treino, y_treino, silent=True)
  else:
    mod = clf.fit(x_treino, y_treino)

  y_prev = mod.predict(x_teste)
  acc = accuracy_score(y_teste, y_prev)

  # Marcação de tempo de fim
  fim = time.time()
  print(nomes_classificadores[idx], ': Accurácia', acc, ': Tempo de execução',fim-inicio,'segundos')

XGBoost : Accurácia 0.8653039268423883 : Tempo de execução 0.722804069519043 segundos
LightGBM : Accurácia 0.9193114577729963 : Tempo de execução 0.2863729000091553 segundos
CatBoost : Accurácia 0.8746637977407208 : Tempo de execução 0.18216753005981445 segundos


(28512, 18)